In [1]:
import numpy as np
import sys
import tensorflow as tf
import os
from threading import Thread
from datetime import datetime
import cv2
import shutil
# from utils import label_map_util
from collections import defaultdict
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from PIL import Image
from PIL import ImageFile

In [2]:
detection_graph = tf.Graph()
sys.path.append("..")

In [3]:
MODEL_NAME = 'drive/MyDrive/hand_inference_graph'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(MODEL_NAME, 'hand_label_map.pbtxt')

NUM_CLASSES = 8

In [4]:
PARENT_PATH = os.getcwd()
DATA_PATH = os.path.join(os.getcwd(),"CombinedNew", "Combined New")
TEST_PATH = os.path.join(DATA_PATH, "test")
TRAIN_PATH = os.path.join(DATA_PATH, "train")
NEW_TRAIN_PATH = os.path.join(DATA_PATH, "new_train")
VALID_PATH = os.path.join(DATA_PATH, "valid")
UNSEEN_PATH = os.path.join(DATA_PATH, "unseen")
MODEL_PATH = "drive/MyDrive/models"
NUM_CLASSES = 8
CLS_LABELS = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7']
SAVE_PATH = os.path.join(os.getcwd(), 'Combined new hand')

In [5]:
if os.path.exists(DATA_PATH):
  shutil.rmtree(DATA_PATH)
if os.path.exists(SAVE_PATH):
  shutil.rmtree(SAVE_PATH)
from zipfile import ZipFile
password = "57Rk!F8yUz38"
with ZipFile('drive/MyDrive/CombinedNew.zip') as zipObj:
  zipObj.extractall()

In [6]:
for class_folder in os.listdir(TRAIN_PATH):
  class_path = os.path.join(TRAIN_PATH, class_folder)
  if class_folder == ".DS_Store":
      continue
  train_imgs, test_imgs = train_test_split(os.listdir(class_path), test_size=0.2, random_state = 42)

  for img in train_imgs: 
    src = os.path.join(class_path, img)
    dst = os.path.join(NEW_TRAIN_PATH, class_folder, img)
    if not os.path.exists(os.path.join(NEW_TRAIN_PATH, class_folder)):
      os.makedirs(os.path.join(NEW_TRAIN_PATH, class_folder))
    shutil.copy(src, dst)

  for img in test_imgs: 
    src = os.path.join(class_path, img)
    dst = os.path.join(VALID_PATH, class_folder, img)
    if not os.path.exists(os.path.join(VALID_PATH, class_folder)):
      os.makedirs(os.path.join(VALID_PATH, class_folder))
    shutil.copy(src, dst)

In [7]:
# -----------------------------------------------
# Hand Detection
# -----------------------------------------------
# download model and proto buf from hhttps://github.com/molyswu/hand_detection/tree/temp/hand_detection/protos

def load_inference_graph():

    # load frozen tensorflow model into memory
    print("> ====== loading HAND frozen graph into memory")
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def = tf.compat.v1.GraphDef()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
        sess = tf.compat.v1.Session(graph=detection_graph)
    print("> ======= Hand Inference graph loaded.")
    return detection_graph, sess


In [8]:
def detect_objects(image_np, detection_graph, sess):
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    image_np_expanded = np.expand_dims(image_np, axis=0)

    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores,
            detection_classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})
    return np.squeeze(boxes), np.squeeze(scores)

In [9]:
detection_graph, sess = load_inference_graph()

> ====== loading HAND frozen graph into memory
> ======= Hand Inference graph loaded.


In [10]:
datasets = [NEW_TRAIN_PATH, VALID_PATH, TEST_PATH, UNSEEN_PATH]
datasets_name = ['new_train', 'valid', 'test', 'unseen']

In [11]:
for dataset in datasets_name:
  for cls_label in CLS_LABELS:
    if not os.path.exists(os.path.join(SAVE_PATH, dataset, cls_label)):
       os.makedirs(os.path.join(SAVE_PATH, dataset, cls_label))

In [12]:
img_paths = []

for i in range(4):
  dataset = datasets[i]
  dataset_name = datasets_name[i]
  for cls_label in CLS_LABELS:
    cls_path = os.path.join(dataset, cls_label)
    for img in os.listdir(cls_path):
      if (img != '.DS_Store'):
        img_path = os.path.join(cls_path, img)
        img_np = np.array(Image.open(img_path))
        # Run object detection
        boxes, scores = detect_objects(img_np, detection_graph, sess)
        ori_img = Image.open(img_path)
        width, height = ori_img.size
        # Create a blank white image with the same dimensions as the original image
        new_image = Image.new("RGB", (width, height), (255, 255, 255))

        for i, box in enumerate(boxes):
          # Get the box coordinates
          ymin, xmin, ymax, xmax = box
          ymin = int(ymin * img_np.shape[0])
          xmin = int(xmin * img_np.shape[1])
          ymax = int(ymax * img_np.shape[0])
          xmax = int(xmax * img_np.shape[1])
          if scores[i] > 0.4: 
            box_pixels = ori_img.crop((xmin, ymin, xmax, ymax))
            new_image.paste(box_pixels, (xmin, ymin))
        save_path = os.path.join(SAVE_PATH, dataset_name, cls_label, img)
        if sum(score > 0.4 for score in scores) > 0:
          save_path = os.path.join(SAVE_PATH, dataset_name, cls_label, img)
          new_image.save(save_path)


In [13]:
# for i in range(4):
#   dataset_name = datasets_name[i]
#   for cls_label in CLS_LABELS:
#     cls_path = os.path.join(SAVE_PATH, dataset_name, cls_label)
#     if not os.path.exists(os.path.join('drive/MyDrive', "Combined hand", dataset, cls_label)):
#       os.makedirs(os.path.join('drive/MyDrive', "Combined hand",  dataset, cls_label))
#     for img in os.listdir(cls_path):
#       src = os.path.join(cls_path, img)
#       des = os.path.join('drive/MyDrive', "Combined hand",  dataset, cls_label, img)
#       print(des)
#       shutil.copy(src, dst)
from google.colab import files
import shutil

shutil.make_archive("Combined new hand", 'zip', "Combined new hand")
files.download("Combined new hand.zip")    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>